VGG16 is a convolution neural net (CNN ) architecture which was used to win ILSVR(Imagenet) competition in 2014. It is considered to be one of the excellent vision model architecture till date. Most unique thing about VGG16 is that instead of having a large number of hyper-parameter they focused on having convolution layers of 3x3 filter with a stride 1 and always used same padding and maxpool layer of 2x2 filter of stride 2.

It follows this arrangement of convolution and max pool layers consistently throughout the whole architecture. In the end it has 2 FC(fully connected layers) followed by a softmax for output. The 16 in VGG16 refers to it has 16 layers that have weights. This network is a pretty large network and it has about 138 million (approx) parameters.

![titile](vgg16arch.png)

![titile](vgg.png)

input size- 224*224*3 but we can change the input size based on the quality of the images which we are using in our project.

- 2 convolution layers-> 1 maxpooling layer
- 2 convolution layers-> 1 maxpooling layer
- 3 convolution layers-> 1 maxpooling layer
- 3 convolution layers-> 1 maxpooling layer
- 3 convolution layers-> 1 maxpooling layer
- DENSE(FULLY CONNECTED)
- DENSE(FULLY CONNECTED)
- DENSE(FULLY CONNECTED)


In all the convolution layers we have:
    - filter size=3*3 
    - stride=1
    - padding=same(means our dimensions of the image(height and width) doesn't change)
In all the maxpooling layers we have:
  - filter size=2*2
  - stride=2
  
Formula for maxpooling:
   - ((n+2p-f)/s)+1
   - n=input dimension value, s=stride, p=padding(if same padding then p=0), f=no. of filters
 

In [2]:
# import the libraries as shown below
import os
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt/


In [3]:
train_path = '../input/cifar10-pngs-in-folders/cifar10/train'
valid_path = '../input/cifar10-pngs-in-folders/cifar10/test'

In [4]:
# Import the VGG16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
IMAGE_SIZE = [224, 224]
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 1s 0us/step


In [5]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [6]:
# useful for getting number of output classes
folders = glob(train_path+'/*')

In [7]:
folders

['../input/cifar10-pngs-in-folders/cifar10/train/ship',
 '../input/cifar10-pngs-in-folders/cifar10/train/frog',
 '../input/cifar10-pngs-in-folders/cifar10/train/cat',
 '../input/cifar10-pngs-in-folders/cifar10/train/bird',
 '../input/cifar10-pngs-in-folders/cifar10/train/dog',
 '../input/cifar10-pngs-in-folders/cifar10/train/airplane',
 '../input/cifar10-pngs-in-folders/cifar10/train/truck',
 '../input/cifar10-pngs-in-folders/cifar10/train/automobile',
 '../input/cifar10-pngs-in-folders/cifar10/train/horse',
 '../input/cifar10-pngs-in-folders/cifar10/train/deer']

In [8]:
# our layers - you can add more if you want
x = Flatten()(vgg16.output)

In [9]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [10]:
# view the structure of the model
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [11]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [12]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [13]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 50000 images belonging to 10 classes.


In [14]:
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 10000 images belonging to 10 classes.


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/10
1563/1563 [==============================] - 715s 458ms/step - loss: 1.1967 - accuracy: 0.5939 - val_loss: 1.0685 - val_accuracy: 0.6319
Epoch 2/10
1563/1563 [==============================] - 635s 406ms/step - loss: 0.9881 - accuracy: 0.6686 - val_loss: 0.9091 - val_accuracy: 0.6917
Epoch 3/10
1563/1563 [==============================] - 633s 405ms/step - loss: 0.9248 - accuracy: 0.6907 - val_loss: 1.0096 - val_accuracy: 0.6672
Epoch 4/10
1563/1563 [==============================] - 632s 404ms/step - loss: 0.9019 - accuracy: 0.7003 - val_loss: 0.9167 - val_accuracy: 0.7025
Epoch 5/10
1563/1563 [==============================] - 631s 403ms/step - loss: 0.8574 - accuracy: 0.7150 - val_loss: 0.9820 - val_accuracy: 0.6868
Epoch 6/10
1563/1563 [==============================] - 635s 406ms/step - loss: 0.8445 - accuracy: 0.7189 - val_loss: 0.9120 - val_accuracy: 0.7027
Epoch 7/10
1563/1563 [==============================] - 624s 399ms/step - loss: 0.8278 - accuracy: 0.7243 - val_

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('model_vgg16.h5')